In [3]:
import pandas as pd
import numpy as np

# Load data and drop unnecessary columns
# get all ES_part_X files, 1 up to 5 and place in one df_es dataframe
df_es = pd.DataFrame()
for i in range(1, 6):
    df_temp = pd.read_csv(f'ES_part_{i}.csv')
    df_es = pd.concat([df_es, df_temp], ignore_index=True)

In [4]:
df_es

,Date,Time,Open,Close,Volume
0,09/10/1997,00:01,0.00,0.00,0
1,09/10/1997,00:02,0.00,0.00,0
2,09/10/1997,00:03,0.00,0.00,0
3,09/10/1997,00:04,0.00,0.00,0
4,09/10/1997,00:05,0.00,0.00,0
...,...,...,...,...,...
9682630,12/19/2024,15:56,5941.75,5941.75,318
9682631,12/19/2024,15:57,5941.75,5941.50,386
9682632,12/19/2024,15:58,5941.50,5941.00,484
9682633,12/19/2024,15:59,5940.75,5941.00,6462


In [21]:
# Create a minute_return column from the Close column
df_es['minute_return'] = df_es['Close'].pct_change()
df_es.dropna(inplace=True)
# drop minute_return with 'inf' as value
df_es = df_es.replace([np.inf, -np.inf], np.nan)
df_es

sumstatcsv = pd.read_csv('ES_part_1.csv')
sumstatcsv.to_csv('sumstatES.csv', index=False)

Summary Statistics for the S&P500 futures file

In [25]:
import pandas as pd
import numpy as np
import re # Kept for the cleaning function, although likely not needed for this specific file

# Define the input and output filenames
output_csv_file = 'summary_statistics_ES.csv'

def clean_numeric(value):
    """
    Cleans a string value to convert it into a numeric type.
    Handles characters like 'k', 'm', 'b', '$', '%'.
    Returns NaN if conversion is not possible.
    (Note: This might be overkill for sumstatES.csv but kept for robustness)
    """
    if isinstance(value, (int, float)):
        return value
    if not isinstance(value, str):
        return np.nan

    value = value.strip()
    multiplier = 1

    # Remove currency symbols and commas
    value = re.sub(r'[$,]', '', value)

    # Handle percentage sign
    if '%' in value:
        value = value.replace('%', '')
        multiplier *= 0.01 # Convert percentage to decimal

    # Handle 'k', 'm', 'b' suffixes
    if 'k' in value:
        value = value.replace('k', '')
        multiplier *= 1000
    elif 'm' in value:
        value = value.replace('m', '')
        multiplier *= 1000000
    elif 'b' in value:
        value = value.replace('b', '')
        multiplier *= 1000000000

    try:
        # Attempt conversion to float
        numeric_value = float(value)
        return numeric_value * multiplier
    except ValueError:
        # Return NaN if conversion fails
        return np.nan

try:
    # Read the CSV file into a pandas DataFrame
    # No obvious index column in the data itself
    df = df_es

    # Identify potential numeric columns (excluding Date/Time)
    potential_numeric_cols = ['Open', 'Close', 'Volume']

    # Apply cleaning/conversion to potential numeric columns
    for col in potential_numeric_cols:
        if col in df.columns:
            # Use pd.to_numeric for efficiency if complex cleaning isn't expected
            df[col] = pd.to_numeric(df[col], errors='coerce')
            # Alternatively, keep using clean_numeric if needed:
            # df[col] = df[col].apply(clean_numeric)


    # Select only columns that are now numeric
    df_numeric = df.select_dtypes(include=np.number)

    # Calculate summary statistics (count, mean, median, std, min, max)
    summary_stats = df_numeric.agg(['count', 'mean', 'median', 'std', 'min', 'max'])

    # Transpose the table so that original columns are rows and statistics are columns
    summary_stats_transposed = summary_stats.T

    # Export the transposed summary statistics to a new CSV file
    summary_stats_transposed.to_csv(output_csv_file)

    print(f"Summary statistics calculated successfully for '{df_es}'.")
    print(f"Numeric columns considered: {list(df_numeric.columns)}")
    print(f"Results exported to '{output_csv_file}'")

except FileNotFoundError:
    print(f"Error: The file '{df_es}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

Summary statistics calculated successfully for '               Date   Time     Open    Close  Volume  minute_return
7        09/10/1997  00:08  1176.75  1176.75       0       0.000000
8        09/10/1997  00:09  1176.75  1176.75       0       0.000000
9        09/10/1997  00:10  1176.75  1176.75       0       0.000000
10       09/10/1997  00:11  1176.75  1176.75       0       0.000000
11       09/10/1997  00:12  1176.75  1176.75       0       0.000000
...             ...    ...      ...      ...     ...            ...
9682630  12/19/2024  15:56  5941.75  5941.75     318       0.000000
9682631  12/19/2024  15:57  5941.75  5941.50     386      -0.000042
9682632  12/19/2024  15:58  5941.50  5941.00     484      -0.000084
9682633  12/19/2024  15:59  5940.75  5941.00    6462       0.000000
9682634  12/19/2024  16:00  5941.00  5941.25    8864       0.000042

[9682628 rows x 6 columns]'.
Numeric columns considered: ['Open', 'Close', 'Volume', 'minute_return']
Results exported to 'summary_stat

In [6]:
# get summary statistics for the df_es dataframe
# give start and ending date, number of observations, mean minute return, median minute return, st dev of returns, min and max of returns, mean and median volume and std dev of volume
start_date = df_es['Date'].min()
end_date = df_es['Date'].max()
num_observations = len(df_es)
mean_minute_return = df_es['minute_return'].mean()
median_minute_return = df_es['minute_return'].median()
stdev_minute_return = df_es['minute_return'].std()
min_minute_return = df_es['minute_return'].min()
max_minute_return = df_es['minute_return'].max()
mean_volume = df_es['Volume'].mean()
median_volume = df_es['Volume'].median()
stdev_volume = df_es['Volume'].std()
summary_stats = {
    'Start Date': start_date,
    'End Date': end_date,
    'Number of Observations': num_observations,
    'Mean Minute Return': mean_minute_return,
    'Median Minute Return': median_minute_return,
    'St Dev of Returns': stdev_minute_return,
    'Min of Returns': min_minute_return,
    'Max of Returns': max_minute_return,
    'Mean Volume': mean_volume,
    'Median Volume': median_volume,
    'St Dev of Volume': stdev_volume
}

summary_stats_df = pd.DataFrame(summary_stats, index=[0])
summary_stats_df

,Start Date,End Date,Number of Observations,Mean Minute Return,Median Minute Return,St Dev of Returns,Min of Returns,Max of Returns,Mean Volume,Median Volume,St Dev of Volume
0,01/01/1998,12/31/2021,9682629,2.249018e-07,0.0,0.00034,-0.053767,0.030826,864.230206,91.0,2243.275253


<h1>Summary statistics for Surprises</h1>

In [7]:
df_surprise = pd.read_csv('US_economic_releases_events.csv')
df_surprise


C:\Users\Jop Brouwer\AppData\Local\Temp\ipykernel_28072\2152678328.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_surprise = pd.read_csv('US_economic_releases_events.csv')


,Unnamed: 0,Period,Event,Ticker,Actual,Prior,Revised,S,Freq.,First Rev.,...,Month,Surv(A),Surv(H),Surv(L),Surv(M),# Ests.,Std Dev,Surprise,Country/Region,Flag
0,1997-01-02,Dec,ISM Manufacturing,NAPMPMI Index,55.2,53,--,94.81480,M,--,...,Jan,--,--,--,--,0.0,--,--,United States,United States
1,1997-01-03,Nov,Construction Spending,VNCCTOT Index,1.9$,1.8$,1.5$,0.00000,M,1.5$,...,Jan,-0.40$,--,--,-0.40$,0.0,--,--,United States,United States
2,1997-01-06,Dec,Housing Completions,PHUCCHNG Index,2k,-1k,--,0.00000,M,--,...,Jan,--,--,--,--,0.0,--,--,United States,United States
3,1997-01-07,Nov,Factory Orders,TMNOCHNG Index,-0.004,0.009,0.011,84.44440,M,0.011,...,Jan,-0.007,--,--,-0.007,0.0,--,--,United States,United States
4,1997-01-08,Dec,Consumer Credit,CICRTOT Index,$7.400b,$2.200b,--,42.96300,M,--,...,Jan,$3.60b,--,--,$3.60b,0.0,--,--,United States,United States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36053,2024-12-31,Oct,FHFA House Price Index MoM,HPIMMOM% Index,--,0.007,--,68.88890,M,--,...,Dec,--,--,--,--,0.0,--,--,United States,United States
36054,2024-12-31,Oct,S&P CoreLogic CS 20-City MoM SA,SPCS20SM Index,--,0.0018,--,30.37040,M,--,...,Dec,--,--,--,--,1.0,--,--,United States,United States
36055,2024-12-31,Oct,S&P CoreLogic CS 20-City YoY NSA,SPCS20Y% Index,--,0.0457,--,40.74070,M,--,...,Dec,--,--,--,--,1.0,--,--,United States,United States
36056,2024-12-31,Oct,S&P CoreLogic CS US HPI YoY NSA,SPCSUSAY Index,--,0.0389,--,35.55560,M,--,...,Dec,--,--,--,--,0.0,--,--,United States,United States


In [8]:
# Dropping all rows for which surprise column has NaN
df_surprise.dropna(subset=['Surprise'], inplace=True)

df_surprise.replace("--", pd.NA, inplace=True)

# Convert 'Surprise' column to float
df_surprise['Surprise'] = pd.to_numeric(df_surprise['Surprise'], errors='coerce')

# Again filtering out rows where 'Surprise' is 0 or NaN
df_surprise = df_surprise[df_surprise['Surprise'] != 0].dropna(subset=['Surprise'])

df_surprise

# Redoing dropping all rows for which surprise column has NaN or 0 value
df_surprise.dropna(subset=['Surprise'], inplace=True)

df_surprise.dropna(subset=['Time'], inplace=True)

# Wincorsizing to get results between 0.5% and 99.5% percentile for Surprise values

lower_bound = df_surprise['Surprise'].quantile(0.005)
upper_bound = df_surprise['Surprise'].quantile(0.995)

df_surprise = df_surprise[(df_surprise['Surprise'] >= lower_bound) & (df_surprise['Surprise'] <= upper_bound)]

df_surprise.to_csv('US_economic_releases_events_forsummarystats.csv', index=False)



C:\Users\Jop Brouwer\AppData\Local\Temp\ipykernel_28072\2502497980.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_surprise.replace("--", pd.NA, inplace=True)


In [9]:
# Create summary statistics for the df_surprise dataframe
# Give total number of observations, number of positive surprises, number of negative surprises, mean surprise, median surprise, st dev of surprise, min and max of surprise, number of different Event, average surprise
total_observations = len(df_surprise)
num_positive_surprises = len(df_surprise[df_surprise['Surprise'] > 0])
num_negative_surprises = len(df_surprise[df_surprise['Surprise'] < 0])
mean_surprise = df_surprise['Surprise'].mean()
median_surprise = df_surprise['Surprise'].median()
stdev_surprise = df_surprise['Surprise'].std()
min_surprise = df_surprise['Surprise'].min()
max_surprise = df_surprise['Surprise'].max()
num_different_events = df_surprise['Event'].nunique()
average_surprise = df_surprise['Surprise'].mean()
summary_stats_surprise = {
    'Total Observations': total_observations,
    'Number of Positive Surprises': num_positive_surprises,
    'Number of Negative Surprises': num_negative_surprises,
    'Mean Surprise': mean_surprise,
    'Median Surprise': median_surprise,
    'St Dev of Surprise': stdev_surprise,
    'Min of Surprise': min_surprise,
    'Max of Surprise': max_surprise,
    'Number of Different Events': num_different_events,
    'Average Surprise': average_surprise
}  
summary_stats_surprise_df = pd.DataFrame(summary_stats_surprise, index=[0])
summary_stats_surprise_df

,Total Observations,Number of Positive Surprises,Number of Negative Surprises,Mean Surprise,Median Surprise,St Dev of Surprise,Min of Surprise,Max of Surprise,Number of Different Events,Average Surprise
0,23169,11577,11592,0.027621,-0.01,2.551963,-9.58,10.58,158,0.027621


In [10]:
# column names for df_surprise print out
df_surprise.columns.tolist()


['Unnamed: 0',
 'Period',
 'Event',
 'Ticker',
 'Actual',
 'Prior',
 'Revised',
 'S',
 'Freq.',
 'First Rev.',
 'Last Rev.',
 'Date',
 'Time',
 'C',
 'Category',
 'Subcategory',
 'R',
 'Day',
 'Month',
 'Surv(A)',
 'Surv(H)',
 'Surv(L)',
 'Surv(M)',
 '# Ests.',
 'Std Dev',
 'Surprise',
 'Country/Region',
 'Flag']

In [20]:
# print out summary statistics csv
sstatt = pd.read_csv('US_economic_releases_events_forsummarystats.csv')
sstatt

,Unnamed: 0,Period,Event,Ticker,Actual,Prior,Revised,S,Freq.,First Rev.,...,Month,Surv(A),Surv(H),Surv(L),Surv(M),# Ests.,Std Dev,Surprise,Country/Region,Flag
0,1997-06-25,1Q F,GDP Annualized QoQ,GDP CQOQ Index,0.059,0.058,NaN,96.296300,Q,NaN,...,Jun,0.0583,0.063,0.055,0.058,13.0,0.22,0.45,United States,United States
1,1997-08-01,Jul,Change in Nonfarm Payrolls,NFP TCH Index,316k,217k,228k,99.259300,M,228k,...,Aug,66.50k,135k,-5k,70k,20.0,33.84,7.27,United States,United States
2,1997-12-23,3Q F,GDP Annualized QoQ,GDP CQOQ Index,0.031,0.033,NaN,96.296300,Q,NaN,...,Dec,0.0335,0.035,0.033,0.033,11.0,0.07,-2.98,United States,United States
3,1998-03-26,4Q F,GDP Annualized QoQ,GDP CQOQ Index,0.037,0.039,NaN,96.296300,Q,NaN,...,Mar,0.0391,0.041,0.037,0.039,22.0,0.08,-2.52,United States,United States
4,1998-04-30,1Q A,GDP Annualized QoQ,GDP CQOQ Index,0.042,0.037,NaN,96.296300,Q,NaN,...,Apr,0.0334,0.041,0.027,0.034,19.0,0.36,2.22,United States,United States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23164,2024-12-18,Nov P,Building Permits,NHSPATOT Index,1505k,1416k,1419k,61.185200,M,1419k,...,Dec,1432.83k,1552k,1390k,1430k,46.0,22.21,3.38,United States,United States
23165,2024-12-18,Nov P,Building Permits MoM,NHCHATCH Index,0.061,-0.006,-0.004,31.851900,M,-0.004,...,Dec,0.0107,0.096,-0.047,0.01,46.0,1.78,2.87,United States,United States
23166,2024-12-18,3Q,Current Account Balance,USCABAL Index,-$310.9b,-$266.8b,-$275.0b,71.111100,Q,-$275.0b,...,Dec,-$287.04b,-$276.4b,-$300.0b,-$287.1b,18.0,6.22,-3.84,United States,United States
23167,2024-12-18,2024-12-18 00:00:00,FOMC Median Rate Forecast: Next Yr,DOTDY1MD Index,0.03875,0.03375,NaN,2.222220,D,NaN,...,Dec,0.036,0.03875,0.03375,0.03625,12.0,0.12,2.03,United States,United States


In [15]:
import pandas as pd
import numpy as np
import re

# Define the input and output filenames
input_csv_file = 'US_economic_releases_events_forsummarystats.csv'
output_csv_file = 'summary_statistics.csv'

def clean_numeric(value):
    """
    Cleans a string value to convert it into a numeric type.
    Handles characters like 'k', 'm', 'b', '$', '%'.
    Returns NaN if conversion is not possible.
    """
    if isinstance(value, (int, float)):
        return value
    if not isinstance(value, str):
        return np.nan

    value = value.strip()
    multiplier = 1

    # Remove currency symbols and commas
    value = re.sub(r'[$,]', '', value)

    # Handle percentage sign
    if '%' in value:
        value = value.replace('%', '')
        multiplier *= 0.01 # Convert percentage to decimal

    # Handle 'k', 'm', 'b' suffixes
    if 'k' in value:
        value = value.replace('k', '')
        multiplier *= 1000
    elif 'm' in value:
        value = value.replace('m', '')
        multiplier *= 1000000
    elif 'b' in value:
        value = value.replace('b', '')
        multiplier *= 1000000000

    try:
        # Attempt conversion to float
        numeric_value = float(value)
        return numeric_value * multiplier
    except ValueError:
        # Return NaN if conversion fails
        return np.nan

try:
    # Read the CSV file into a pandas DataFrame
    # Use the first column as the index if it's unnamed and seems like an index
    df = pd.read_csv(input_csv_file, index_col=0)

    # Apply the cleaning function to all columns
    # This is broad but ensures potentially numeric columns are cleaned
    for col in df.columns:
        # Only attempt cleaning if the column isn't purely numeric already
        if not pd.api.types.is_numeric_dtype(df[col]):
             df[col] = df[col].apply(clean_numeric)

    # Select only columns that are now numeric
    df_numeric = df.select_dtypes(include=np.number)

    # Calculate summary statistics (mean, median, std, min, max)
    summary_stats = df_numeric.agg(['mean', 'median', 'std', 'min', 'max'])

    # Transpose the table so that original columns are rows and statistics are columns
    summary_stats_transposed = summary_stats.T

    # Export the transposed summary statistics to a new CSV file
    summary_stats_transposed.to_csv(output_csv_file)

    print(f"Summary statistics calculated successfully.")
    print(f"Cleaned numeric columns considered: {list(df_numeric.columns)}")
    print(f"Results exported to '{output_csv_file}'")

except FileNotFoundError:
    print(f"Error: The file '{input_csv_file}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

Summary statistics calculated successfully.
Cleaned numeric columns considered: ['Period', 'Event', 'Ticker', 'Actual', 'Prior', 'Revised', 'S', 'Freq.', 'First Rev.', 'Last Rev.', 'Date', 'Time', 'C', 'Category', 'Subcategory', 'R', 'Day', 'Month', 'Surv(A)', 'Surv(H)', 'Surv(L)', 'Surv(M)', '# Ests.', 'Std Dev', 'Surprise', 'Country/Region', 'Flag']
Results exported to 'summary_statistics.csv'


c:\Users\Jop Brouwer\Documents\GitHub\Data-Science-Seminar-Group-Project-5\.venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\Jop Brouwer\Documents\GitHub\Data-Science-Seminar-Group-Project-5\.venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\Jop Brouwer\Documents\GitHub\Data-Science-Seminar-Group-Project-5\.venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\Jop Brouwer\Documents\GitHub\Data-Science-Seminar-Group-Project-5\.venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\Jop Brouwer\Documents\GitHub\Data-Science-Seminar-Group-Project-5\.venv

In [26]:
df_surprise

,Event,Ticker,Prior,Revised,Freq.,First Rev.,Last Rev.,Time,R,Surv(M),# Ests.,Surprise,DateTime
153,GDP Annualized QoQ,GDP CQOQ Index,0.058,<NA>,Q,<NA>,<NA>,07:30:00,96.296300,0.058,13.0,0.45,1997-06-25 07:30:00
198,Change in Nonfarm Payrolls,NFP TCH Index,217k,228k,M,228k,265k,07:30:00,99.259300,70k,20.0,7.27,1997-08-01 07:30:00
394,GDP Annualized QoQ,GDP CQOQ Index,0.033,<NA>,Q,<NA>,0.051,07:30:00,96.296300,0.033,11.0,-2.98,1997-12-23 07:30:00
523,GDP Annualized QoQ,GDP CQOQ Index,0.039,<NA>,Q,<NA>,<NA>,07:30:00,96.296300,0.039,22.0,-2.52,1998-03-26 07:30:00
569,GDP Annualized QoQ,GDP CQOQ Index,0.037,<NA>,Q,<NA>,<NA>,07:30:00,96.296300,0.034,19.0,2.22,1998-04-30 07:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35992,Building Permits,NHSPATOT Index,1416k,1419k,M,1419k,1419k,07:30:00,61.185200,1430k,46.0,3.38,2024-12-18 07:30:00
35993,Building Permits MoM,NHCHATCH Index,-0.006,-0.004,M,-0.004,-0.004,07:30:00,31.851900,0.01,46.0,2.87,2024-12-18 07:30:00
35994,Current Account Balance,USCABAL Index,-$266.8b,-$275.0b,Q,-$275.0b,-$275.0b,07:30:00,71.111100,-$287.1b,18.0,-3.84,2024-12-18 07:30:00
35999,FOMC Median Rate Forecast: Next Yr,DOTDY1MD Index,0.03375,<NA>,D,<NA>,<NA>,13:00:00,2.222220,0.03625,12.0,2.03,2024-12-18 13:00:00


In [29]:
# print out number of unique events
df_surprise['Event'].nunique()

158